In [16]:
%pylab
from itertools import product
import numpy as np
from pylab import *  # plotting library 
from numba import jit
import matplotlib.animation as am
import random
import time

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Square Lattice Monte Carlo

In [17]:
def generate_sublattice_sq(L):
    index_r_list_sq = []
    index_b_list_sq = []

    if (L % 2) == 0:
        m = int(L/2)
        n = int(L/2)
    else:
        m = int((L+1)/2)
        n = int((L-1)/2)

    for i in range(L):
        if (i % 2) == 0:
            for k in range(m):
                index_r_list_sq.append((i, 2*k))
            for k in range(n):
                index_b_list_sq.append((i, 2*k+1))
        else:
            for k in range(m):
                index_b_list_sq.append((i, 2*k))
            for k in range(n):
                index_r_list_sq.append((i, 2*k+1))
        
    return np.array(index_r_list_sq), np.array(index_b_list_sq)

## Antiferromagnetic Case

In [18]:
#create the sublattices
r, b = generate_sublattice_sq(50)
lr = len(r)
lb = len(b)

print(lr, lb)

@jit(nopython = True)
def MCstep_jit_2_AFM(N, L, T, h, state, acceptedMoves, energy, mag_r, mag_b):
    
    randomArray = np.random.random(N)
    
    for k in range(N):

        rb = np.random.randint(2)
        if rb == 0:
            coord = np.random.randint(lr)
            c = r[coord]
        else:
            coord = np.random.randint(lb)
            c = b[coord]
            
        i = c[0]
        j = c[1]
        
        dE = -2*state[i, j] * (state[(i+1)%L, j] + state[i-1, j] + state[i, (j+1)%L] + state[i, j-1]) + h*state[i,j]
        
        if dE <= 0 or np.exp(-dE/T) > randomArray[k]:
            
            #keep track of moves
            acceptedMoves += 1
            
            #update the spin
            newSpin = -state[i, j] # flip spin
            state[i, j] = newSpin
            
            #update the energy
            energy += dE
            
            #update the magnetizations
            if rb == 0:
                mag_r += 2*newSpin
            else:
                mag_b += 2*newSpin

    return state, acceptedMoves, energy, mag_r, mag_b

1250 1250


In [51]:
class Ising2D_2_AFM (object):

    def __init__(self, L = 32, temperature = 10.0, field = 0):
        
        self.L = L
        self.N = L**2
        self.acceptedMoves = 0
        
        self.temperature = temperature
        self.field = field
        
        self.state = ones((self.L, self.L), int) # initially all spins up
        self.energy = 2*self.N
        self.magnetization_r = self.N/2
        self.magnetization_b = self.N/2
        self.magnetization = 0
        
        self.reset()
        
    def increment_T(self, T_increment, reset = True):

        T_new = self.temperature + T_increment
        
        if T_new <= 0:
            T_new = self.temperature

        self.temperature = T_new
        if reset:
            self.reset()    
            
    def reset(self):

        self.monteCarloSteps = 0
        self.acceptedMoves = 0
        self.energyArray = array([], int)
        self.magnetizationArray = array([], int)
        self.magnetizationArray_r = array([], int)
        self.magnetizationArray_b = array([], int)            
            
    def monteCarloStep(self):

        N = self.N
        L = self.L
        T = self.temperature
        h = self.field
        
        state = self.state
        acceptedMoves = self.acceptedMoves
        energy = self.energy
        mag_r = self.magnetization_r
        mag_b = self.magnetization_b

        state, acceptedMoves, energy, magnetization_r, magnetization_b = MCstep_jit_2_AFM(N, L, T, h, state, acceptedMoves, energy, mag_r, mag_b)
        
        self.state = state
        self.acceptedMoves = acceptedMoves
        self.energy = energy
        self.magnetization_r = magnetization_r
        self.magnetization_b = magnetization_b
        self.magnetization = magnetization_r - magnetization_b
        
        self.energyArray.append(self.energy)
        self.magnetizationArray.append(self.magnetization)
        self.magnetizationArray_r.append(self.magnetization_r)
        self.magnetizationArray_b.append(self.magnetization_b)
        self.monteCarloSteps += 1

      
    def steps(self, number = 100):

        self.energyArray = self.energyArray.tolist()
        self.magnetizationArray = self.magnetizationArray.tolist()
        self.magnetizationArray_r = self.magnetizationArray_r.tolist() 
        self.magnetizationArray_b = self.magnetizationArray_b.tolist() 
        
        for i in range(number):
            self.monteCarloStep()

        self.energyArray = np.asarray(self.energyArray)
        self.magnetizationArray = np.asarray(self.magnetizationArray)
        self.magnetizationArray_r = np.asarray(self.magnetizationArray_r) 
        self.magnetizationArray_b = np.asarray(self.magnetizationArray_b)
                
    # Observables
    def mE(self):
        return self.energyArray.mean() / self.N

    def Cp(self):
        return (self.energyArray.std() / self.temperature)**2 / self.N
            
    def mMr(self):
        return self.magnetizationArray_r.mean() / self.N
    
    def mMb(self):
        return self.magnetizationArray_b.mean() / self.N
    
    def mM(self):
        return self.magnetizationArray.mean() / self.N
    
    def Xr(self):
        return (self.magnetizationArray_r.std())**2 / (self.temperature * self.N)
    
    def Xb(self):
        return (self.magnetizationArray_b.std())**2 / (self.temperature * self.N)
    
    def X(self):
        return (self.magnetizationArray.std())**2 / (self.temperature * self.N)


In [52]:
def T_dep_2_AFM(init_temp, length, dt):
    
    t_incr = -dt
    T = init_temp
    model = Ising2D_2_AFM(temperature = T, L = length)
    
    #thermo variables
    Cp = []
    E = []
    Temp = []
    
    #blue and red susceptibilities
    Xr = []
    Xb = []
    X = []
    
    #blue and red magnetizations
    Mr = []
    Mb = []
    M = []

    while T > 0.5 :

        model.steps(number = 1000)
        model.reset()
        model.steps(number = 30000)
        
        #thermo variables
        Cp.append(model.Cp())
        E.append(model.mE())
        Temp.append(T)
        
        #blue and red susceptibilities
        Xr.append(model.Xr())
        Xb.append(model.Xb())
        X.append(model.X())
        
        #blue and red magnetizations
        Mr.append(model.mMr())
        Mb.append(model.mMb())
        M.append(model.mM())
        
        #progress the temperature
        model.increment_T(t_incr)
        T = model.temperature

    return Temp, Cp, E, Xr, Xb, X, Mr, Mb, M

## Ferromagnetic Case

In [43]:
#create the sublattices
r, b = generate_sublattice_sq(50)
lr = len(r)
lb = len(b)

print(lr, lb)

@jit(nopython = True)
def MCstep_jit_2_FM(N, L, T, h, state, acceptedMoves, energy, mag_r, mag_b):
    
    randomArray = np.random.random(N)
    
    for k in range(N):

        rb = np.random.randint(2)
        if rb == 0:
            coord = np.random.randint(lr)
            c = r[coord]
        else:
            coord = np.random.randint(lb)
            c = b[coord]
            
        i = c[0]
        j = c[1]
        
        dE = 2*state[i, j] * (state[(i+1)%L, j] + state[i-1, j] + state[i, (j+1)%L] + state[i, j-1]) + h*state[i,j]
        
        if dE <= 0 or np.exp(-dE/T) > randomArray[k]:
            
            #keep track of moves
            acceptedMoves += 1
            
            #update the spin
            newSpin = -state[i, j] # flip spin
            state[i, j] = newSpin
            
            #update the energy
            energy += dE
            
            #update the magnetizations
            if rb == 0:
                mag_r += 2*newSpin
            else:
                mag_b += 2*newSpin

    return state, acceptedMoves, energy, mag_r, mag_b

1250 1250


In [44]:
class Ising2D_2_FM (object):

    def __init__(self, L = 32, temperature = 10.0, field = 0):
        
        self.L = L
        self.N = L**2
        self.acceptedMoves = 0
        
        self.temperature = temperature
        self.field = field
        
        self.state = ones((self.L, self.L), int) # initially all spins up
        self.energy = -2*self.N
        self.magnetization_r = self.N/2
        self.magnetization_b = self.N/2
        self.magnetization = self.N
        
        self.reset()
        
    def increment_T(self, T_increment, reset = True):

        T_new = self.temperature + T_increment
        
        if T_new <= 0:
            T_new = self.temperature

        self.temperature = T_new
        if reset:
            self.reset()    
            
    def reset(self):

        self.monteCarloSteps = 0
        self.acceptedMoves = 0
        self.energyArray = array([], int)
        self.magnetizationArray = array([], int)
        self.magnetizationArray_r = array([], int)
        self.magnetizationArray_b = array([], int)            
            
    def monteCarloStep(self):

        N = self.N
        L = self.L
        T = self.temperature
        h = self.field
        
        state = self.state
        acceptedMoves = self.acceptedMoves
        energy = self.energy
        mag_r = self.magnetization_r
        mag_b = self.magnetization_b

        state, acceptedMoves, energy, magnetization_r, magnetization_b = MCstep_jit_2_FM(N, L, T, h, state, acceptedMoves, energy, mag_r, mag_b)
        
        self.state = state
        self.acceptedMoves = acceptedMoves
        self.energy = energy
        self.magnetization_r = magnetization_r
        self.magnetization_b = magnetization_b
        self.magnetization = magnetization_r + magnetization_b
        
        self.energyArray.append(self.energy)
        self.magnetizationArray.append(self.magnetization)
        self.magnetizationArray_r.append(self.magnetization_r)
        self.magnetizationArray_b.append(self.magnetization_b)
        self.monteCarloSteps += 1

      
    def steps(self, number = 100):

        self.energyArray = self.energyArray.tolist()
        self.magnetizationArray = self.magnetizationArray.tolist()
        self.magnetizationArray_r = self.magnetizationArray_r.tolist() 
        self.magnetizationArray_b = self.magnetizationArray_b.tolist() 
        
        for i in range(number):
            self.monteCarloStep()

        self.energyArray = np.asarray(self.energyArray)
        self.magnetizationArray = np.asarray(self.magnetizationArray)
        self.magnetizationArray_r = np.asarray(self.magnetizationArray_r) 
        self.magnetizationArray_b = np.asarray(self.magnetizationArray_b)
                
    # Observables
    def mE(self):
        return self.energyArray.mean() / self.N

    def Cp(self):
        return (self.energyArray.std() / self.temperature)**2 / self.N
            
    def mMr(self):
        return self.magnetizationArray_r.mean() / self.N
    
    def mMb(self):
        return self.magnetizationArray_b.mean() / self.N
    
    def mM(self):
        return self.magnetizationArray.mean() / self.N
    
    def Xr(self):
        return (self.magnetizationArray_r.std())**2 / (self.temperature * self.N)
    
    def Xb(self):
        return (self.magnetizationArray_b.std())**2 / (self.temperature * self.N)
    
    def X(self):
        return (self.magnetizationArray.std())**2 / (self.temperature * self.N)


In [45]:
def T_dep_2_FM(init_temp, length, dt):
    
    t_incr = -dt
    T = init_temp
    model = Ising2D_2_FM(temperature = T, L = length)
    
    #thermo variables
    Cp = []
    E = []
    Temp = []
    
    #blue and red susceptibilities
    Xr = []
    Xb = []
    X = []
    
    #blue and red magnetizations
    Mr = []
    Mb = []
    M = []

    while T > 0.5 :

        model.steps(number = 1000)
        model.reset()
        model.steps(number = 30000)
        
        #thermo variables
        Cp.append(model.Cp())
        E.append(model.mE())
        Temp.append(T)
        
        #blue and red susceptibilities
        Xr.append(model.Xr())
        Xb.append(model.Xb())
        X.append(model.X())
        
        #blue and red magnetizations
        Mr.append(model.mMr())
        Mb.append(model.mMb())
        M.append(model.mM())
        
        #progress the temperature
        model.increment_T(t_incr)
        T = model.temperature

    return Temp, Cp, E, Xr, Xb, X, Mr, Mb, M

## Results

In [53]:
start = time.time()

#T_fm, Cp_fm, E_fm, Xr_fm, Xb_fm, X_fm, Mr_fm, Mb_fm, M_fm = T_dep_2_FM(6, 50, 0.05)
T_afm, Cp_afm, E_afm, Xr_afm, Xb_afm, X_afm, Mr_afm, Mb_afm, M_afm = T_dep_2_AFM(6, 50, 0.05)

end = time.time()
print(end-start)

384.4967439174652


In [55]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(T_afm, X_afm, label = "Staggered Susceptibility (AFM)")
ax.scatter(T_fm, X_fm, label = "Suscpetibility (FM)")
ax.set_xlabel("Temperature", fontsize = 20)
ax.set_ylabel("Susceptibility", fontsize = 20)
ax.set_title("Suscpetibility and Staggered Susceptibility of Ising spins on a Square Lattice (Monte Carlo)", fontsize = 30)
ax.legend(fontsize = 15)

In [50]:
fig = plt.figure()

ax = []
for i in range(4):
    ax_temp = fig.add_subplot(2,2,i+1)
    ax.append(ax_temp)

ax[0].scatter(T_afm, Xr_afm, label = "Red Sublattice AFM")
ax[0].scatter(T_afm, Xb_afm, label = "Blue Sublattice AFM")
ax[1].scatter(T_afm, Mr_afm, label = "Red Sublattice AFM")
ax[1].scatter(T_afm, Mb_afm, label = "Blue Sublattice AFM")
ax[2].scatter(T_afm, Cp_afm, label = "AFM")
ax[3].scatter(T_afm, E_afm, label = "AFM")

#ax[0].scatter(T_fm, Xr_fm)
ax[0].scatter(T_fm, X_fm, label = "FM")
ax[1].scatter(T_fm, M_fm, label = "FM")
#ax[1].scatter(T_fm, Mr_fm, label = "Red Sublattice FM")
#ax[1].scatter(T_fm, Mb_fm, label = "Blue Sublattice FM")
ax[2].scatter(T_fm, Cp_fm, label = "FM")
ax[3].scatter(T_fm, E_fm, label = "FM")

ax[0].set_xlabel("Temperature", fontsize = 20)
ax[1].set_xlabel("Temperature", fontsize = 20)
ax[2].set_xlabel("Temperature", fontsize = 20)
ax[3].set_xlabel("Temperature", fontsize = 20)

ax[0].set_ylabel("Susceptibility", fontsize = 20)
ax[1].set_ylabel("Magnetization", fontsize = 20)
ax[2].set_ylabel("Specific Heat", fontsize = 20)
ax[3].set_ylabel("Mean Energy", fontsize = 20)

for i in range(4):
    ax[i].legend(fontsize = 15)

fig.suptitle("Ferro- and Antiferromagnetic Ising Spins (Square Lattice, Monte Carlo)", fontsize = 25)

Text(0.5, 0.98, 'Ferro- and Antiferromagnetic Ising Spins (Square Lattice, Monte Carlo)')